In [2]:
from datasets import Dataset
import pandas as pd

data_json = pd.read_json("../data/preprocessed/covid_test")
data_json

,text,label,hashtags,emojis,polarity,subjectivity,sentiment
0,post claim compulsory vaccination violates pri...,0,[],[trade_mark],0.033333,0.500000,positive
1,photo claim person doctor died attending covid...,0,[],[],0.500000,0.500000,positive
2,post video claim protest confiscation town ama...,0,[],[],0.000000,0.000000,neutral
3,death respiratory failure pneumonia registered...,0,[],[],-0.316667,0.300000,negative
4,dean college biologist adieu state lot cpr fal...,0,[],[trade_mark],-0.400000,0.600000,negative
5,household covid patient porto allegra campos g...,0,[],[],0.000000,0.000000,neutral
6,chain list recommendation prevent treat corona...,0,[],[],0.000000,0.000000,neutral
7,60000 argentinian company closed due covid com...,0,[],[],-0.112500,0.237500,negative
8,social medium post criticize photo smiling pre...,0,[],[],0.033333,0.066667,positive
9,cdc released update coronavirus transmitted,0,[],[],0.000000,0.000000,neutral


In [9]:
import pyarrow as pa

schema = pa.schema([
    ("text", pa.string()),
    ("label", pa.int64()),
    ("hashtags", pa.list_(pa.string())),
    ("emojis", pa.list_(pa.string())),
    ("polarity", pa.float64()),
    ("subjectivity", pa.float64()),
    ("sentiment", pa.string()),
])

arrow_table = pa.Table.from_batches([], schema=schema)

dataset = Dataset(arrow_table=arrow_table)
dataset = dataset.from_pandas(data_json)

[['trade_mark'],
 [],
 [],
 [],
 ['trade_mark'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [10]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_19', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [12]:
def tokenize_dataset(dataset):
    return tokenizer(dataset["text"])

In [13]:
dataset = dataset.map(tokenize_dataset)
tf_dataset = model.prepare_tf_dataset(
    dataset, batch_size=16, shuffle=True, tokenizer=tokenizer
)

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [14]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(3e-5))
model.fit(tf_dataset)  # doctest: +SKIP

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
1/1 [==============================] - 17s 17s/step - loss: 0.7033
